In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [33]:
import json
import openai
from psy_llm_chat import LlamaChat, GptChat
from datetime import datetime   
import pytz
from typing import Dict, List, Tuple
import glob
import random
from copy import deepcopy

In [5]:
def get_now(
    local_tz: str="America/New_York",
    dt_format: str="%d-%m-%Y—%H:%M:%S—%Z"
) -> str:
    # datetime object containing current date and time
    local = pytz.timezone(local_tz)
    naive = datetime.now()
    local_dt = local.localize(naive, is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime(dt_format)

def make_res(
    result: List[Dict],
    exp_name: str,
    model_name: str
) -> Tuple[str,Dict]:
    now = get_now()
    save_res = {"experiment_name": exp_name,
                "model_name": model_name,
                "run_date": now,
                "dialog": result}
    save_name = f"{exp_name}_{model_name}_{now}.txt"
    return save_name, save_res

In [67]:
def get_question(diagram):
    msg = deepcopy(messages)
    q = deepcopy(question)
    q["content"] = q["content"].format(diagram)
    msg += [q]
    return msg

def run_exp(cht_cls, fns, diagrams):
    results = {}
    for fn, diagram in zip(fns, diagrams): 
        q = get_question(diagram)
        uneek = fn.split("/")[-1].split(".")[0]
        results[uneek] = cht_cls(q)
    return results


In [52]:
messages =[
    {"role": "user", "content": "Hi!"},
    {"role": "assistant", "content": "Hi, how can I help you today?"},
    {"role": "user", "content": """I'm going to give you some TiKZ code that specifies a pulley system, then I'm going to ask you a question about
        the pulley system, OK?"""},
    {"role": "assistant", "content": "Sounds great, please upload the TiKZ code and ask me about it."}
    ]

question = {"role": "user", "content": """Ok, here is the TiKZ code depicting a pulley system:\n\n```Latex\n\n{}\n```\n\n Now I'm going to ask a question about the pulley system depicted above.
    In answering my question, you can assume that the system is idealized, with no rope stretch and no friction. My question is:\n
    What is the mechanical advantage of the system depicted in the TiKZ code above?"""}

In [53]:
fns = [fn for fn in glob.glob("../../materials/simple_pulleys/*.tex")]
diagrams = [open(fn,"r").read() for fn in fns]
random.shuffle(diagrams)

In [54]:
[fn.split("/")[-1].split(".")[0] for fn in fns]

['simp_MA1_B',
 'simp_MA5_A',
 'simp_MA2_A',
 'simp_MA1_A',
 'simp_MA5_B',
 'simp_MA3_B',
 'simp_MA2_B',
 'simp_MA3_A',
 'simp_MA4_B',
 'simp_MA4_A']

In [69]:
#  print available GPT4 models
#authenticate openai with your key
openai.api_key = os.environ["OPENAI"]

[mod["id"] for mod in openai.Model.list()["data"] if mod["id"].startswith("gpt-4")]

['gpt-4-0314', 'gpt-4-0613', 'gpt-4']

In [73]:
GPT_NAME = 'gpt-4-0314'
EXP_NAME = "decide"

In [76]:
gptc = GptChat(model=GPT_NAME)

In [ ]:
gpt_results = run_exp(gptc, fns, diagrams)

In [ ]:
gpt_name, gpt_res = make_res(gpt_results, EXP_NAME, GPT_NAME) 
gpt_name, gpt_res

In [63]:
with open(f"./{gpt_name}","w") as f:
    json.dump(gpt_res,f,indent=4)

In [64]:
#  #local models 
# model_name ="./llama_models/models_hf/7B_chat"
# model_name ="./llama_models/models_hf/13B_chat"
LLAMA_NAME ="../../llama_models/models_hf/70B_chat"

# # huggingface
# models can also be loaded from HF hub (once access has been granted), e.g.:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "meta-llama/Llama-2-13b-chat-hf"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

In [65]:
messages

[{'role': 'user', 'content': 'Hi!'},
 {'role': 'assistant', 'content': 'Hi, how can I help you today?'},
 {'role': 'user',
  'content': "I'm going to give you some TiKZ code that specifies a pulley system, then I'm going to ask you a question about\n        the pulley system, OK?"},
 {'role': 'assistant',
  'content': 'Sounds great, please upload the TiKZ code and ask me about it.'}]

In [66]:
# load the model defined by <<model_name>>, must set output_hidden_states and ...attentions to True to access these later.
lc = LlamaChat(LLAMA_NAME, True, max_new_tokens=1024, output_hidden_states=True, output_attentions=True)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
llama_results = run_exp(lc, fns, diagrams)

In [112]:
llama_name, llama_res = make_res(llama_results,EXP_NAME,LLAMA_NAME.split("/")[-1]) 
llama_name 

pprint(llama_results["simp_MA5_A"])

[{'content': 'Hi!', 'role': 'user'},
 {'content': 'Hi, how can I help you today?', 'role': 'assistant'},
 {'content': "I'm going to give you some TiKZ code that specifies a pulley "
             "system, then I'm going to ask you a question about\n"
             '        the pulley system, OK?',
  'role': 'user'},
 {'content': 'Sounds great, please upload the TiKZ code and ask me about it.',
  'role': 'assistant'},
 {'content': 'Ok, here is the TiKZ code depicting a pulley system:\n'
             '\n'
             '```Latex\n'
             '\n'
             '\\documentclass{article}\n'
             '\\usepackage{caption}\n'
             '\\usepackage{subcaption}\n'
             '\\usepackage{graphicx}\n'
             '\\usepackage{tikz}\n'
             '\\usepackage{tikzsymbols}\n'
             '\\usetikzlibrary{calc}\n'
             '\\usepackage{float}\n'
             '\\usepackage{pdflscape}\n'
             '\\usepackage{geometry}\n'
             '\\geometry{a4paper, landscape, marg

In [85]:
with open(f"./{llama_name}","w") as f:
    json.dump(llama_res,f,indent=4)

In [107]:
# !pip install openpyxl
import pandas as pd
df = pd.concat([pd.DataFrame([{"uneek": key, 
                          "generated_result": val[-2]["content"],
                          "model": llama_res["model_name"]} for key,val in llama_res["dialog"].items()]),
           pd.DataFrame([{"uneek": key,
                          "generated_result": 
                          val[-2]["content"], 
                          "model": gpt_res["model_name"]} for key,val in gpt_res["dialog"].items()])
          ]
         )
df.to_excel("./decide_result.xlsx",index=False)
df.to_csv("./decide_result.csv",index=False)